### Question 1: IPO Filings Web Scraping and Data Processing

**What's the total sum ($m) of 2023 filings that happenned of Fridays?**

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/
Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs).
Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given).
You may be inspired by the function `extract_numbers()` in [Code Snippet 4], or you can write your own function to "parse" a string.
Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all fillings during 2023, which happened on Fridays (`Date.dt.dayofweek()==4`). You should see 32 records in total, 24 of it is not null.

(additional: you can read about [S-1 IPO filing](https://www.dfinsolutions.com/knowledge-hub/thought-leadership/knowledge-resources/what-s-1-ipo-filing) to understand the context)

In [1]:
!pip install pandas

In [2]:
import pandas as pd
import requests

import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date, datetime


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)
ipo_dfs

[      Filing Date Symbol                           Company Name  \
 0     May 3, 2024    TBN         Tamboran Resources Corporation   
 1    Apr 29, 2024   HWEC                   HW Electro Co., Ltd.   
 2    Apr 29, 2024   DTSQ  DT Cloud Star Acquisition Corporation   
 3    Apr 26, 2024   EURK                Eureka Acquisition Corp   
 4    Apr 26, 2024    HDL    Super Hi International Holding Ltd.   
 ..            ...    ...                                    ...   
 319  Jan 21, 2020   GOXS                            Goxus, Inc.   
 320  Jan 21, 2020   UTXO                 UTXO Acquisition, Inc.   
 321   Dec 9, 2019   LOHA                           Loha Co. Ltd   
 322   Oct 4, 2019   ZGHB  China Eco-Materials Group Co. Limited   
 323  Dec 27, 2018   FBOX              Fit Boxx Holdings Limited   
 
         Price Range Shares Offered  
 0                 -              -  
 1             $3.00        3750000  
 2            $10.00        6000000  
 3            $10.00        50

In [3]:
#check datatypes of columns
ipo_dfs[0].info()
ipos_dfs = ipo_dfs[0]
ipos_dfs

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     324 non-null    object
 1   Symbol          324 non-null    object
 2   Company Name    324 non-null    object
 3   Price Range     324 non-null    object
 4   Shares Offered  324 non-null    object
dtypes: object(5)
memory usage: 12.8+ KB


,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
...,...,...,...,...,...
319,"Jan 21, 2020",GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000
320,"Jan 21, 2020",UTXO,"UTXO Acquisition, Inc.",$10.00,5000000
321,"Dec 9, 2019",LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000
322,"Oct 4, 2019",ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000


In [4]:
# convert datatype of IPO Date column from string to datetime
ipos_dfs['Filing Date'] = pd.to_datetime(ipos_dfs['Filing Date'], format="%b %d, %Y")
ipos_dfs
#stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format="%b %d, %Y")

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-05-03,TBN,Tamboran Resources Corporation,-,-
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,-
...,...,...,...,...,...
319,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000
320,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000
321,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000
322,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000


In [5]:
ipos_dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     324 non-null    datetime64[ns]
 1   Symbol          324 non-null    object        
 2   Company Name    324 non-null    object        
 3   Price Range     324 non-null    object        
 4   Shares Offered  324 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 12.8+ KB


In [6]:
ipos_dfs['Shares Offered'] = pd.to_numeric(ipos_dfs['Shares Offered'].str.replace('-', ' '), errors='coerce')
ipos_dfs

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN
...,...,...,...,...,...
319,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0
320,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0
321,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0
322,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0


In [7]:
ipos_dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     324 non-null    datetime64[ns]
 1   Symbol          324 non-null    object        
 2   Company Name    324 non-null    object        
 3   Price Range     324 non-null    object        
 4   Shares Offered  250 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 12.8+ KB


In [8]:
ipos_dfs.isnull().sum() #of null values in dataframe

Filing Date        0
Symbol             0
Company Name       0
Price Range        0
Shares Offered    74
dtype: int64

In [9]:
import re

def extract_numbers(input_string):
    split_string = input_string.split(" - ")
    y_match = re.search(r'(\d+.\d)', split_string[0])
    if len(split_string) > 1:
      m_match = re.search(r'(\d+.\d)', split_string[1])
      y1_number, m1_number = float(y_match.group(1)) if y_match else 0, float(m_match.group(1)) if m_match else 0
      return (y1_number + m1_number)/len(split_string)
    else:
      y0 = float(y_match.group(1)) if y_match else 0
      return y0


In [10]:
ipos_dfs['Avg_price'] = ipos_dfs['Price Range'].apply(lambda x: extract_numbers(x))
display(ipos_dfs)

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,0.00
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.00
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.00
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.00
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,0.00
...,...,...,...,...,...,...
319,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0,9.00
320,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0,10.00
321,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0,9.00
322,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0,4.00


In [11]:
ipos_dfs['Shares_offered_value'] = ipos_dfs['Shares Offered'] * ipos_dfs['Avg_price']
ipos_dfs

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,0.00,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.00,11250000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.00,60000000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.00,50000000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,0.00,NaN
...,...,...,...,...,...,...,...
319,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0,9.00,13500000.0
320,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0,10.00,50000000.0
321,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0,9.00,22500000.0
322,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0,4.00,17200000.0


In [12]:
ipos_dfs.info()
#ipos_dfs.set_index('Filing Date')
df = ipos_dfs

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Filing Date           324 non-null    datetime64[ns]
 1   Symbol                324 non-null    object        
 2   Company Name          324 non-null    object        
 3   Price Range           324 non-null    object        
 4   Shares Offered        250 non-null    float64       
 5   Avg_price             324 non-null    float64       
 6   Shares_offered_value  250 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 17.8+ KB


In [13]:
#extract subset if record filing in 2023

df1 = df.loc[df['Filing Date'] >= '2023-01-01']
df2 = df1.loc[df1['Filing Date'] <= '2023-12-31']
df2 = df2.reset_index(drop=True)
df2

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
0,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000.0,4.0,4800000.0
1,2023-12-29,EPSM,Epsium Enterprise Limited,-,NaN,0.0,NaN
2,2023-12-28,ONDR,"Sushi Ginza Onodera, Inc.",$7.00 - $8.00,1066667.0,7.5,8000002.5
3,2023-12-22,LZMH,LZ Technology Holdings Limited,-,NaN,0.0,NaN
4,2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,-,NaN,0.0,NaN
...,...,...,...,...,...,...,...
112,2023-01-31,FBGL,FBS Global Limited,$4.00 - $5.00,1875000.0,4.5,8437500.0
113,2023-01-24,THNK,"T1V, Inc.",$4.00 - $6.00,3300000.0,5.0,16500000.0
114,2023-01-23,RPET,New Ruipeng Pet Group Inc.,-,NaN,0.0,NaN
115,2023-01-13,RVGO,"RVeloCITY, Inc.",$4.00 - $5.00,3750000.0,4.5,16875000.0


In [14]:
def numofday(dt):
  num = dt.weekday()
  return num

In [15]:
#get num of week day
df2['Day of week'] = df2['Filing Date'].apply(lambda x: numofday(x))
df2

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value,Day of week
0,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000.0,4.0,4800000.0,4
1,2023-12-29,EPSM,Epsium Enterprise Limited,-,NaN,0.0,NaN,4
2,2023-12-28,ONDR,"Sushi Ginza Onodera, Inc.",$7.00 - $8.00,1066667.0,7.5,8000002.5,3
3,2023-12-22,LZMH,LZ Technology Holdings Limited,-,NaN,0.0,NaN,4
4,2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,-,NaN,0.0,NaN,4
...,...,...,...,...,...,...,...,...
112,2023-01-31,FBGL,FBS Global Limited,$4.00 - $5.00,1875000.0,4.5,8437500.0,1
113,2023-01-24,THNK,"T1V, Inc.",$4.00 - $6.00,3300000.0,5.0,16500000.0,1
114,2023-01-23,RPET,New Ruipeng Pet Group Inc.,-,NaN,0.0,NaN,0
115,2023-01-13,RVGO,"RVeloCITY, Inc.",$4.00 - $5.00,3750000.0,4.5,16875000.0,4


In [16]:
#filter on day = 4
df_3 = df2.loc[df2['Day of week'] == 4] #only extract values of 4 in the day of week column
df_final = df_3[~df_3['Shares_offered_value'].isna()] #select only non null values in the shares offered value column
df_final

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value,Day of week
0,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000.0,4.00,4800000.0,4
11,2023-12-08,ENGS,Energys Group Limited,$4.00 - $6.00,2000000.0,5.00,10000000.0,4
12,2023-12-08,LNKS,Linkers Industries Limited,$4.00 - $6.00,2200000.0,5.00,11000000.0,4
31,2023-10-27,RAY,Raytech Holding Limited,$4.00 - $5.00,1500000.0,4.50,6750000.0,4
38,2023-10-13,ORIS,Oriental Rise Holdings Limited,$4.00,2000000.0,4.00,8000000.0,4
41,2023-10-06,QMMM,QMMM Holdings Limited,$4.00,2125000.0,4.00,8500000.0,4
45,2023-09-29,KAPA,"Kairos Pharma, Ltd.",$4.00,1550000.0,4.00,6200000.0,4
46,2023-09-29,VAPA,Valens Pay Global Limited,$5.00 - $6.00,1000000.0,5.50,5500000.0,4
52,2023-09-15,ACSB,Acesis Holdings Corporation,$4.00 - $6.00,1300000.0,5.00,6500000.0,4
71,2023-07-07,AZI,Autozi Internet Technology (Global) Ltd.,$4.00 - $5.00,1250000.0,4.50,5625000.0,4


In [17]:
total = df_final['Shares_offered_value'].sum()
f"total sum in millions: ${round(total/1000000)}M"


'total sum in millions: $286M'

### Question 2:  IPOs "Fixed days hold" strategy


**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**


Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/).
Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks.
Some of the tickers like 'DYCQ' and 'LEGT' were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day).
Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:
* You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
* You can use the `DataFrame.describe()` function to get mean, min, max, 25-50-75% quantiles.


Addtional:
* You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
* What's your recommendation: Do you suggest pursuing this strategy for an optimal X?

In [18]:
url = "https://stockanalysis.com/ipos/2023/"
url2 = "https://stockanalysis.com/ipos/2024/"

def get_ipo_df(string):
  response = requests.get(string, headers=headers)
  df = pd.read_html(response.text)
  return df[0]

df_2023, df_2024 = get_ipo_df(url), get_ipo_df(url2)




In [19]:
display(df_2023)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 27, 2023",IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.40%
1,"Dec 19, 2023",LGCB,Linkage Global Inc,$4.00,$2.86,-27.25%
2,"Dec 15, 2023",ZKH,ZKH Group Limited,$15.50,$12.64,-21.16%
3,"Dec 15, 2023",BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,"Dec 14, 2023",INHD,Inno Holdings Inc.,$4.00,$0.64,-84.05%
...,...,...,...,...,...,...
149,"Jan 25, 2023",QSG,QuantaSing Group Ltd,$12.50,$3.35,-74.80%
150,"Jan 20, 2023",CVKD,"Cadrenal Therapeutics, Inc.",$5.00,$0.49,-90.40%
151,"Jan 13, 2023",SKWD,"Skyward Specialty Insurance Group, Inc.",$15.00,$37.07,146.03%
152,"Jan 13, 2023",ISRL,Israel Acquisitions Corp,$10.00,$10.93,9.30%


In [20]:
display(df_2024)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 9, 2024",SVCO,"Silvaco Group, Inc.",$19.00,$19.79,7.68%
1,"May 9, 2024",PAL,"Proficient Auto Logistics, Inc.",$15.00,$14.88,-0.90%
2,"May 8, 2024",NNE,Nano Nuclear Energy Inc.,$4.00,$4.72,34.75%
3,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$28.52,19.54%
4,"Apr 26, 2024",ZONE,"CleanCore Solutions, Inc.",$4.00,$3.12,-20.75%
...,...,...,...,...,...,...
62,"Jan 18, 2024",PSBD,Palmer Square Capital BDC Inc.,$16.45,$16.65,1.37%
63,"Jan 18, 2024",CCTG,CCSC Technology International Holdings Limited,$6.00,$2.77,-54.17%
64,"Jan 12, 2024",SYNX,Silynxcom Ltd.,$4.00,$2.58,-35.50%
65,"Jan 11, 2024",SDHC,Smith Douglas Homes Corp.,$21.00,$29.03,38.83%


In [21]:
#stacking dataframes
stacked_ipos_df = pd.concat([df_2024, df_2023], ignore_index=True)
stacked_ipos_df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 9, 2024",SVCO,"Silvaco Group, Inc.",$19.00,$19.79,7.68%
1,"May 9, 2024",PAL,"Proficient Auto Logistics, Inc.",$15.00,$14.88,-0.90%
2,"May 8, 2024",NNE,Nano Nuclear Energy Inc.,$4.00,$4.72,34.75%
3,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$28.52,19.54%
4,"Apr 26, 2024",ZONE,"CleanCore Solutions, Inc.",$4.00,$3.12,-20.75%
...,...,...,...,...,...,...
216,"Jan 25, 2023",QSG,QuantaSing Group Ltd,$12.50,$3.35,-74.80%
217,"Jan 20, 2023",CVKD,"Cadrenal Therapeutics, Inc.",$5.00,$0.49,-90.40%
218,"Jan 13, 2023",SKWD,"Skyward Specialty Insurance Group, Inc.",$15.00,$37.07,146.03%
219,"Jan 13, 2023",ISRL,Israel Acquisitions Corp,$10.00,$10.93,9.30%


In [22]:
# convert datatype of IPO Date column from string to datetime

stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format="%b %d, %Y")


In [23]:
#apply the filter date
df_filtered = stacked_ipos_df.loc[stacked_ipos_df['IPO Date']< '2024-03-01']
df_filtered

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
36,2024-02-27,SMXT,"SolarMax Technology, Inc.",$4.00,$10.99,171.25%
37,2024-02-22,VHAI,Vocodia Holdings Corp,$4.25,$0.14,-96.73%
38,2024-02-21,DYCQ,DT Cloud Acquisition Corporation,$10.00,$10.16,1.60%
39,2024-02-16,CHRO,Chromocell Therapeutics Corp,$6.00,$1.75,-70.83%
40,2024-02-14,UMAC,"Unusual Machines, Inc.",$4.00,$1.11,-73.75%
...,...,...,...,...,...,...
216,2023-01-25,QSG,QuantaSing Group Ltd,$12.50,$3.35,-74.80%
217,2023-01-20,CVKD,"Cadrenal Therapeutics, Inc.",$5.00,$0.49,-90.40%
218,2023-01-13,SKWD,"Skyward Specialty Insurance Group, Inc.",$15.00,$37.07,146.03%
219,2023-01-13,ISRL,Israel Acquisitions Corp,$10.00,$10.93,9.30%


In [24]:
stacked_copy = df_filtered
#df = pd.DataFrame(columns=['Date'])
df_copy = stacked_copy.set_index('IPO Date')
display(df_copy)

,Symbol,Company Name,IPO Price,Current,Return
IPO Date,,,,,
2024-02-27,SMXT,"SolarMax Technology, Inc.",$4.00,$10.99,171.25%
2024-02-22,VHAI,Vocodia Holdings Corp,$4.25,$0.14,-96.73%
2024-02-21,DYCQ,DT Cloud Acquisition Corporation,$10.00,$10.16,1.60%
2024-02-16,CHRO,Chromocell Therapeutics Corp,$6.00,$1.75,-70.83%
2024-02-14,UMAC,"Unusual Machines, Inc.",$4.00,$1.11,-73.75%
...,...,...,...,...,...
2023-01-25,QSG,QuantaSing Group Ltd,$12.50,$3.35,-74.80%
2023-01-20,CVKD,"Cadrenal Therapeutics, Inc.",$5.00,$0.49,-90.40%
2023-01-13,SKWD,"Skyward Specialty Insurance Group, Inc.",$15.00,$37.07,146.03%


In [25]:
new_list = []
for stock in df_copy['Symbol']:
  if stock in ['IBAC', 'PTHR', 'BKHA']:
    stock+='U'
    new_list.append(stock)
  elif stock == 'SBXC':
    stock+='-UN'
    new_list.append(stock)
  else:
    new_list.append(stock)

In [26]:
new_list.sort() # sort list of amended tickers

In [27]:
new_list

['AACT',
 'AESI',
 'AFJK',
 'AHR',
 'AITR',
 'AIXI',
 'ALCY',
 'ANL',
 'ANRO',
 'ANSC',
 'APGE',
 'ARBB',
 'ARM',
 'AS',
 'ASST',
 'ATGL',
 'ATMU',
 'ATS',
 'AVBP',
 'AZTR',
 'BANL',
 'BAYA',
 'BFRG',
 'BIRK',
 'BLAC',
 'BMR',
 'BOF',
 'BOWN',
 'BREA',
 'BTSG',
 'BUJA',
 'CART',
 'CAVA',
 'CCTG',
 'CETU',
 'CGON',
 'CHRO',
 'CHSN',
 'CLBR',
 'CRGX',
 'CTNT',
 'CVKD',
 'CWD',
 'DDC',
 'DIST',
 'DTCK',
 'DYCQ',
 'ELAB',
 'ELWS',
 'ENLT',
 'ESHA',
 'FBLG',
 'FEBO',
 'FIHL',
 'FORL',
 'FTEL',
 'GDHG',
 'GDTC',
 'GENK',
 'GLAC',
 'GMM',
 'GNLX',
 'GODN',
 'GPAK',
 'GPCR',
 'GSIW',
 'GUTS',
 'GVH',
 'GXAI',
 'HAO',
 'HG',
 'HKIT',
 'HLP',
 'HLXB',
 'HRYU',
 'HSAI',
 'HSHP',
 'HYAC',
 'ICG',
 'INHD',
 'INTS',
 'IPXX',
 'IROH',
 'ISPR',
 'ISRL',
 'IVP',
 'IZM',
 'JL',
 'JNVR',
 'JVSA',
 'JYD',
 'KGS',
 'KSPI',
 'KVAC',
 'KVUE',
 'KVYO',
 'KYTX',
 'LEGT',
 'LGCB',
 'LICN',
 'LQR',
 'LRE',
 'LRHC',
 'LSDI',
 'LXEO',
 'MARX',
 'MDBH',
 'MGIH',
 'MGOL',
 'MGRX',
 'MGX',
 'MIRA',
 'MLYS',
 'MNR',
 

In [28]:
len(new_list), len(set(new_list)) #test for duplicates

(185, 185)

In [29]:
merged_df = []

for ticker in new_list:
  df_ticker = yf.download(tickers = ticker,
                        period = "max",
                        interval = "1d")
  df_ticker['Symbol'] = ticker
  df_ticker["adj_close_minus_1"] = df_ticker['Adj Close'].shift(-1)
  df_ticker["adj_close_growth_1d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_1"] - 1
  #df_ticker = df_ticker.drop("adj_close_minus_1", axis=1)
  df_ticker["adj_close_minus_2"] = df_ticker['Adj Close'].shift(-2)
  df_ticker["adj_close_growth_2d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_2"] - 1
  df_ticker["adj_close_minus_3"] = df_ticker['Adj Close'].shift(-3)
  df_ticker["adj_close_growth_3d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_3"] - 1
  df_ticker["adj_close_minus_4"] = df_ticker['Adj Close'].shift(-4)
  df_ticker["adj_close_growth_4d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_4"] - 1
  df_ticker["adj_close_minus_5"] = df_ticker['Adj Close'].shift(-5)
  df_ticker["adj_close_growth_5d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_5"] - 1
  df_ticker["adj_close_minus_6"] = df_ticker['Adj Close'].shift(-6)
  df_ticker["adj_close_growth_6d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_6"] - 1
  df_ticker["adj_close_minus_7"] = df_ticker['Adj Close'].shift(-7)
  df_ticker["adj_close_growth_7d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_7"] - 1
  df_ticker["adj_close_minus_8"] = df_ticker['Adj Close'].shift(-8)
  df_ticker["adj_close_growth_8d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_8"] - 1
  df_ticker["adj_close_minus_9"] = df_ticker['Adj Close'].shift(-9)
  df_ticker["adj_close_growth_9d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_9"] - 1
  df_ticker["adj_close_minus_10"] = df_ticker['Adj Close'].shift(-10)
  df_ticker["adj_close_growth_10d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_10"] - 1
  df_ticker["adj_close_minus_11"] = df_ticker['Adj Close'].shift(-11)
  df_ticker["adj_close_growth_11d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_11"] - 1
  df_ticker["adj_close_minus_12"] = df_ticker['Adj Close'].shift(-12)
  df_ticker["adj_close_growth_12d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_12"] - 1
  df_ticker["adj_close_minus_13"] = df_ticker['Adj Close'].shift(-13)
  df_ticker["adj_close_growth_13d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_13"] - 1
  df_ticker["adj_close_minus_14"] = df_ticker['Adj Close'].shift(-14)
  df_ticker["adj_close_growth_14d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_14"] - 1
  df_ticker["adj_close_minus_15"] = df_ticker['Adj Close'].shift(-15)
  df_ticker["adj_close_growth_15d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_15"] - 1
  df_ticker["adj_close_minus_16"] = df_ticker['Adj Close'].shift(-16)
  df_ticker["adj_close_growth_16d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_16"] - 1
  #df_ticker = df_ticker.drop("adj_close_minus_1", axis=1)
  df_ticker["adj_close_minus_17"] = df_ticker['Adj Close'].shift(-17)
  df_ticker["adj_close_growth_17d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_17"] - 1
  df_ticker["adj_close_minus_18"] = df_ticker['Adj Close'].shift(-18)
  df_ticker["adj_close_growth_18d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_18"] - 1
  df_ticker["adj_close_minus_19"] = df_ticker['Adj Close'].shift(-19)
  df_ticker["adj_close_growth_19d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_19"] - 1
  df_ticker["adj_close_minus_20"] = df_ticker['Adj Close'].shift(-20)
  df_ticker["adj_close_growth_20d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_20"] - 1
  df_ticker["adj_close_minus_21"] = df_ticker['Adj Close'].shift(-21)
  df_ticker["adj_close_growth_21d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_21"] - 1
  df_ticker["adj_close_minus_22"] = df_ticker['Adj Close'].shift(-22)
  df_ticker["adj_close_growth_22d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_22"] - 1
  df_ticker["adj_close_minus_23"] = df_ticker['Adj Close'].shift(-23)
  df_ticker["adj_close_growth_23d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_23"] - 1
  df_ticker["adj_close_minus_24"] = df_ticker['Adj Close'].shift(-24)
  df_ticker["adj_close_growth_24d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_24"] - 1
  df_ticker["adj_close_minus_25"] = df_ticker['Adj Close'].shift(-25)
  df_ticker["adj_close_growth_25d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_25"] - 1
  df_ticker["adj_close_minus_26"] = df_ticker['Adj Close'].shift(-26)
  df_ticker["adj_close_growth_26d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_26"] - 1
  df_ticker["adj_close_minus_27"] = df_ticker['Adj Close'].shift(-27)
  df_ticker["adj_close_growth_27d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_27"] - 1
  df_ticker["adj_close_minus_28"] = df_ticker['Adj Close'].shift(-28)
  df_ticker["adj_close_growth_28d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_28"] - 1
  df_ticker["adj_close_minus_29"] = df_ticker['Adj Close'].shift(-29)
  df_ticker["adj_close_growth_29d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_14"] - 1
  df_ticker["adj_close_minus_30"] = df_ticker['Adj Close'].shift(-30)
  df_ticker["adj_close_growth_30d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_30"] - 1
  merged_df.append(df_ticker)
#df_ticker['adj_close_minus_2'] = df_ticker['Adj Close'].shift(-2)
#appended_data = pd.concat(merged_df)
#appended_data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [30]:
df_final = pd.concat(merged_df, axis=0, ignore_index=False)
display(df_final)
#df1 = df.loc[df['IPO Date'] >= '2023-01-01']

,Open,High,Low,Close,Adj Close,Volume,Symbol,adj_close_minus_1,adj_close_growth_1d,adj_close_minus_2,...,adj_close_minus_26,adj_close_growth_26d,adj_close_minus_27,adj_close_growth_27d,adj_close_minus_28,adj_close_growth_28d,adj_close_minus_29,adj_close_growth_29d,adj_close_minus_30,adj_close_growth_30d
Date,,,,,,,,,,,,,,,,,,,,,
2023-06-13,10.14,10.14,10.110,10.110,10.110,5500.0,AACT,10.100,0.000990,10.110,...,10.17,-0.005900,10.17,-0.005900,10.19,-0.007851,10.19,-0.005900,10.19,-0.007851
2023-06-14,10.11,10.13,10.095,10.100,10.100,143900.0,AACT,10.110,-0.000989,10.110,...,10.17,-0.006883,10.19,-0.008832,10.19,-0.008832,10.19,-0.007859,10.18,-0.007859
2023-06-15,10.11,10.11,10.100,10.110,10.110,2061100.0,AACT,10.110,0.000000,10.120,...,10.19,-0.007851,10.19,-0.007851,10.19,-0.007851,10.18,-0.005900,10.19,-0.007851
2023-06-16,10.12,10.12,10.110,10.110,10.110,251000.0,AACT,10.120,-0.000988,10.120,...,10.19,-0.007851,10.19,-0.007851,10.18,-0.006876,10.19,-0.003941,10.18,-0.006876
2023-06-20,10.13,10.13,10.110,10.120,10.120,102800.0,AACT,10.120,0.000000,10.120,...,10.19,-0.006869,10.18,-0.005894,10.19,-0.006869,10.18,-0.003937,10.18,-0.005894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-03,12.35,12.57,11.800,12.340,12.340,27800.0,ZKH,12.730,-0.030636,12.950,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-06,12.00,12.73,12.000,12.730,12.730,21800.0,ZKH,12.950,-0.016988,12.220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-07,12.50,12.99,12.110,12.950,12.950,20200.0,ZKH,12.220,0.059738,12.555,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
#remove columns with substring 'adj_close_minus...]
columns_to_be_dropped = df_final.columns[df_final.columns.str.contains('adj_close_minus')]
columns_to_be_dropped

Index(['adj_close_minus_1', 'adj_close_minus_2', 'adj_close_minus_3',
       'adj_close_minus_4', 'adj_close_minus_5', 'adj_close_minus_6',
       'adj_close_minus_7', 'adj_close_minus_8', 'adj_close_minus_9',
       'adj_close_minus_10', 'adj_close_minus_11', 'adj_close_minus_12',
       'adj_close_minus_13', 'adj_close_minus_14', 'adj_close_minus_15',
       'adj_close_minus_16', 'adj_close_minus_17', 'adj_close_minus_18',
       'adj_close_minus_19', 'adj_close_minus_20', 'adj_close_minus_21',
       'adj_close_minus_22', 'adj_close_minus_23', 'adj_close_minus_24',
       'adj_close_minus_25', 'adj_close_minus_26', 'adj_close_minus_27',
       'adj_close_minus_28', 'adj_close_minus_29', 'adj_close_minus_30'],
      dtype='object')

In [32]:
df_final.drop(columns=columns_to_be_dropped, inplace=True)

In [33]:
df_final

,Open,High,Low,Close,Adj Close,Volume,Symbol,adj_close_growth_1d,adj_close_growth_2d,adj_close_growth_3d,...,adj_close_growth_21d,adj_close_growth_22d,adj_close_growth_23d,adj_close_growth_24d,adj_close_growth_25d,adj_close_growth_26d,adj_close_growth_27d,adj_close_growth_28d,adj_close_growth_29d,adj_close_growth_30d
Date,,,,,,,,,,,,,,,,,,,,,
2023-06-13,10.14,10.14,10.110,10.110,10.110,5500.0,AACT,0.000990,0.000000,0.000000,...,-0.005509,-0.004921,-0.004921,-0.004921,-0.005900,-0.005900,-0.005900,-0.007851,-0.005900,-0.007851
2023-06-14,10.11,10.13,10.095,10.100,10.100,143900.0,AACT,-0.000989,-0.000989,-0.001976,...,-0.005905,-0.005905,-0.005905,-0.006883,-0.006883,-0.006883,-0.008832,-0.008832,-0.007859,-0.007859
2023-06-15,10.11,10.11,10.100,10.110,10.110,2061100.0,AACT,0.000000,-0.000988,-0.000988,...,-0.004921,-0.004921,-0.005900,-0.005900,-0.005900,-0.007851,-0.007851,-0.007851,-0.005900,-0.007851
2023-06-16,10.12,10.12,10.110,10.110,10.110,251000.0,AACT,-0.000988,-0.000988,-0.000988,...,-0.004921,-0.005900,-0.005900,-0.005900,-0.007851,-0.007851,-0.007851,-0.006876,-0.003941,-0.006876
2023-06-20,10.13,10.13,10.110,10.120,10.120,102800.0,AACT,0.000000,0.000000,-0.001972,...,-0.004916,-0.004916,-0.004916,-0.006869,-0.006869,-0.006869,-0.005894,-0.006869,-0.003937,-0.005894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-03,12.35,12.57,11.800,12.340,12.340,27800.0,ZKH,-0.030636,-0.047104,0.009820,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-06,12.00,12.73,12.000,12.730,12.730,21800.0,ZKH,-0.016988,0.041735,0.013939,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-07,12.50,12.99,12.110,12.950,12.950,20200.0,ZKH,0.059738,0.031462,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df = df_final.loc[df_final['Symbol'] == 'AACT']
df

,Open,High,Low,Close,Adj Close,Volume,Symbol,adj_close_growth_1d,adj_close_growth_2d,adj_close_growth_3d,...,adj_close_growth_21d,adj_close_growth_22d,adj_close_growth_23d,adj_close_growth_24d,adj_close_growth_25d,adj_close_growth_26d,adj_close_growth_27d,adj_close_growth_28d,adj_close_growth_29d,adj_close_growth_30d
Date,,,,,,,,,,,,,,,,,,,,,
2023-06-13,10.140,10.140,10.110,10.11,10.11,5500.0,AACT,0.000990,0.000000,0.000000,...,-0.005509,-0.004921,-0.004921,-0.004921,-0.005900,-0.005900,-0.005900,-0.007851,-0.005900,-0.007851
2023-06-14,10.110,10.130,10.095,10.10,10.10,143900.0,AACT,-0.000989,-0.000989,-0.001976,...,-0.005905,-0.005905,-0.005905,-0.006883,-0.006883,-0.006883,-0.008832,-0.008832,-0.007859,-0.007859
2023-06-15,10.110,10.110,10.100,10.11,10.11,2061100.0,AACT,0.000000,-0.000988,-0.000988,...,-0.004921,-0.004921,-0.005900,-0.005900,-0.005900,-0.007851,-0.007851,-0.007851,-0.005900,-0.007851
2023-06-16,10.120,10.120,10.110,10.11,10.11,251000.0,AACT,-0.000988,-0.000988,-0.000988,...,-0.004921,-0.005900,-0.005900,-0.005900,-0.007851,-0.007851,-0.007851,-0.006876,-0.003941,-0.006876
2023-06-20,10.130,10.130,10.110,10.12,10.12,102800.0,AACT,0.000000,0.000000,-0.001972,...,-0.004916,-0.004916,-0.004916,-0.006869,-0.006869,-0.006869,-0.005894,-0.006869,-0.003937,-0.005894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-03,10.620,10.640,10.620,10.62,10.62,12200.0,AACT,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-06,10.640,10.640,10.620,10.62,10.62,29300.0,AACT,0.000000,0.000000,-0.000941,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-07,10.630,10.630,10.620,10.62,10.62,35000.0,AACT,0.000000,-0.000941,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df_renamed = df_final.rename_axis('IPO Date')
display(df_renamed)

,Open,High,Low,Close,Adj Close,Volume,Symbol,adj_close_growth_1d,adj_close_growth_2d,adj_close_growth_3d,...,adj_close_growth_21d,adj_close_growth_22d,adj_close_growth_23d,adj_close_growth_24d,adj_close_growth_25d,adj_close_growth_26d,adj_close_growth_27d,adj_close_growth_28d,adj_close_growth_29d,adj_close_growth_30d
IPO Date,,,,,,,,,,,,,,,,,,,,,
2023-06-13,10.14,10.14,10.110,10.110,10.110,5500.0,AACT,0.000990,0.000000,0.000000,...,-0.005509,-0.004921,-0.004921,-0.004921,-0.005900,-0.005900,-0.005900,-0.007851,-0.005900,-0.007851
2023-06-14,10.11,10.13,10.095,10.100,10.100,143900.0,AACT,-0.000989,-0.000989,-0.001976,...,-0.005905,-0.005905,-0.005905,-0.006883,-0.006883,-0.006883,-0.008832,-0.008832,-0.007859,-0.007859
2023-06-15,10.11,10.11,10.100,10.110,10.110,2061100.0,AACT,0.000000,-0.000988,-0.000988,...,-0.004921,-0.004921,-0.005900,-0.005900,-0.005900,-0.007851,-0.007851,-0.007851,-0.005900,-0.007851
2023-06-16,10.12,10.12,10.110,10.110,10.110,251000.0,AACT,-0.000988,-0.000988,-0.000988,...,-0.004921,-0.005900,-0.005900,-0.005900,-0.007851,-0.007851,-0.007851,-0.006876,-0.003941,-0.006876
2023-06-20,10.13,10.13,10.110,10.120,10.120,102800.0,AACT,0.000000,0.000000,-0.001972,...,-0.004916,-0.004916,-0.004916,-0.006869,-0.006869,-0.006869,-0.005894,-0.006869,-0.003937,-0.005894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-03,12.35,12.57,11.800,12.340,12.340,27800.0,ZKH,-0.030636,-0.047104,0.009820,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-06,12.00,12.73,12.000,12.730,12.730,21800.0,ZKH,-0.016988,0.041735,0.013939,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-07,12.50,12.99,12.110,12.950,12.950,20200.0,ZKH,0.059738,0.031462,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_renamed.describe()
#min = minimum value in each numerical column
#std = measure of dispersion of values around the mean
#25% indicates the value below which 25% of the data falls
#50% median - represents the middle value of the dataset
#75% indicates the value below which 75% of the data falls
#max - maximum value in each numerical column

,Open,High,Low,Close,Adj Close,Volume,adj_close_growth_1d,adj_close_growth_2d,adj_close_growth_3d,adj_close_growth_4d,...,adj_close_growth_21d,adj_close_growth_22d,adj_close_growth_23d,adj_close_growth_24d,adj_close_growth_25d,adj_close_growth_26d,adj_close_growth_27d,adj_close_growth_28d,adj_close_growth_29d,adj_close_growth_30d
count,38125.000000,38125.000000,38125.000000,38125.000000,38125.000000,3.812500e+04,37941.000000,37757.000000,37573.000000,37389.000000,...,34263.000000,34080.000000,33897.000000,33714.000000,33531.000000,33348.000000,33165.000000,32982.000000,35549.000000,32616.000000
mean,12.491177,13.112248,12.024705,12.462681,12.434077,6.374197e+05,0.009181,0.024746,0.042526,0.060384,...,0.386760,0.407329,0.427796,0.448721,0.469931,0.491282,0.512952,0.534731,0.246703,0.579153
std,37.975752,56.622566,32.759867,40.585196,40.573815,4.989684e+06,0.225310,1.464977,2.395589,3.060733,...,8.786984,9.033718,9.276287,9.515422,9.751416,9.984324,10.214711,10.442520,6.912367,10.891763
min,0.002000,0.002000,0.002000,0.002000,0.002000,0.000000e+00,-0.853521,-0.856555,-0.868863,-0.884400,...,-0.920851,-0.912979,-0.919916,-0.918599,-0.921017,-0.933445,-0.932351,-0.949123,-0.911746,-0.963233
25%,1.880000,1.983000,1.770000,1.860000,1.860000,5.500000e+03,-0.014599,-0.019733,-0.024561,-0.027473,...,-0.053419,-0.054419,-0.054798,-0.055936,-0.055977,-0.055596,-0.057573,-0.059156,-0.044853,-0.061557
50%,8.070000,8.340000,7.780000,8.040000,8.040000,4.940000e+04,0.000000,0.000000,0.000000,0.000000,...,0.002979,0.003704,0.003828,0.004076,0.004098,0.004955,0.005462,0.005645,0.000966,0.006187
75%,12.120000,12.450000,11.650000,12.086000,12.080000,2.211000e+05,0.022727,0.034286,0.043919,0.052632,...,0.187602,0.195472,0.201613,0.211957,0.217070,0.225870,0.230769,0.237709,0.131111,0.250026
max,3069.000000,7500.000000,875.000000,4318.000000,4318.000000,3.723413e+08,33.875000,278.000001,365.499973,365.499973,...,365.499973,365.499973,365.499973,365.499973,365.499973,365.499973,365.499973,365.499973,365.499973,365.499973


In [37]:
url1 = "https://www.barchart.com/stocks/indices/sp/sp500"


In [38]:
#df_renamed.describe()

#df_n = df_renamed.describe().columns[df_renamed.describe().columns.str.contains('adj_close_growth')
adj25percent, adj50percent, adj75percent = {}, {}, {}
m = df_renamed.describe().to_dict()
for k in m.items():
  if k[0].startswith('adj_close_growth'):
    #print('outer k', k[0])
    #print('value', m[k[0]]['25%'])
    adj25percent[k[0]] = m[k[0]]['25%']
    adj50percent[k[0]] = m[k[0]]['50%']
    adj75percent[k[0]] = m[k[0]]['75%']





In [39]:
def max_value_key(dict):
  max_value_key = max(dict, key=dict.get)
  max_value = dict[max_value_key]
  return (max_value_key, max_value)

In [40]:
#find the key with the maximum value for the 3 dicts
(key25, max25value), (key50, max50value), (key75, max75value)  = max_value_key(adj25percent), max_value_key(adj50percent), max_value_key(adj75percent)
(key25, max25value), (key50, max50value), (key75, max75value)

(('adj_close_growth_1d', -0.014598526172921567),
 ('adj_close_growth_30d', 0.006186834440691236),
 ('adj_close_growth_30d', 0.2500255488596696))

In [41]:
f"for the above IPOs, holding them for a max of 30 days would offer the best return"

'for the above IPOs, holding them for a max of 30 days would offer the best return'

## Question 3: Is Growth Concentrated in the Largest Stocks?

**Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**


Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks
for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

`US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']`

`EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']`

`INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']`

`LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS`
<br/>

Now let's add the top 12-22 stocks (as of end-April 2024):
<br/>

`NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']`

`NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']`

`NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']`

`LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA`

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)

Calculate  `growth_7d` for every stock and every day.
Get the average daily `growth_7d` for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:
| Date   |      ticker_category      |  growth_7d |
|----------|:-------------:|------:|
| 2014-01-01 |  LARGE | 1.011684 |
| 2014-01-01 |   LARGEST   |   1.011797 |

On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)



In [42]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA
LARGE_STOCKS



['PRX.AS',
 'CDI.PA',
 'AIR.PA',
 'SU.PA',
 'ETN',
 'SNY',
 'BUD',
 'DTE.DE',
 'ALV.DE',
 'MDT',
 'AI.PA',
 'EL.PA',
 'TSLA',
 'WMT',
 'XOM',
 'UNH',
 'MA',
 'PG',
 'JNJ',
 'MRK',
 'HD',
 'COST',
 'ORCL',
 'BAJFINANCE.NS',
 'MARUTI.NS',
 'HCLTECH.NS',
 'TATAMOTORS.NS',
 'SUNPHARMA.NS',
 'ONGC.NS',
 'ADANIENT.NS',
 'ADANIENT.NS',
 'NTPC.NS',
 'KOTAKBANK.NS',
 'TITAN.NS']

In [43]:
def get_7d_growth_df(df:pd.DataFrame, i)->pd.DataFrame:
  #for i in [1,3,7,30,90,365]:
  df['growth_'+ str(i)+'d'] = df['Adj Close'] / df['Adj Close'].shift(i)
  GROWTH_KEYS = [k for k in df.keys() if k.startswith('growth')]
  return df[GROWTH_KEYS]

In [44]:
largest_stock_df = []
i = 7
for ticker in LARGEST_STOCKS:
  df_ticker = yf.download(tickers = ticker,
                        period = "max",
                        interval = "1d")
  df_ticker['Symbol'] = ticker
  df_copy = df_ticker.copy()
  df_filtered = df_copy.loc[df_copy.index >= '2013-12-20']
  df = df_filtered.loc[df_filtered.index <= '2023-12-31']
  df['growth_'+ str(i)+'d'] = df['Adj Close'] / df['Adj Close'].shift(i)
  largest_stock_df.append(df)
df_largest = pd.concat(largest_stock_df, axis=0, ignore_index=False)
df_largest




[*********************100%%**********************]  1 of 1 completed
<ipython-input-44-51197be68b30>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['growth_'+ str(i)+'d'] = df['Adj Close'] / df['Adj Close'].shift(i)
[*********************100%%**********************]  1 of 1 completed
<ipython-input-44-51197be68b30>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['growth_'+ str(i)+'d'] = df['Adj Close'] / df['Adj Close'].shift(i)
[*********************100%%**********************]  1 of 1 completed

,Open,High,Low,Close,Adj Close,Volume,Symbol,growth_7d
Date,,,,,,,,
2013-12-20,36.200001,36.930000,36.189999,36.799999,30.930483,62649100,MSFT,NaN
2013-12-23,36.810001,36.889999,36.549999,36.619999,30.779190,25128700,MSFT,NaN
2013-12-24,36.720001,37.169998,36.639999,37.080002,31.165819,14243000,MSFT,NaN
2013-12-26,37.200001,37.490002,37.169998,37.439999,31.468412,17612800,MSFT,NaN
2013-12-27,37.580002,37.619999,37.169998,37.290001,31.342331,14563000,MSFT,NaN
...,...,...,...,...,...,...,...,...
2023-12-22,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707,LT.NS,1.022987
2023-12-26,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263,LT.NS,1.016588
2023-12-27,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266,LT.NS,1.016055


In [45]:
large_stock_df = []
i = 7
for ticker in LARGE_STOCKS:
  df_stocks = yf.download(tickers = ticker,
                        period = "max",
                        interval = "1d")
  df_stocks['Symbol'] = ticker
  df_copy_stocks = df_stocks.copy()
  df_f_stocks = df_copy_stocks.loc[df_copy_stocks.index >= '2013-12-20']
  df_l = df_f_stocks.loc[df_f_stocks.index <= '2024-04-30']
  df_l['growth_'+ str(i)+'d'] = df_l['Adj Close'] / df_l['Adj Close'].shift(i)
  large_stock_df.append(df_l)
df_large = pd.concat(large_stock_df, axis=0, ignore_index=False)
df_large

[*********************100%%**********************]  1 of 1 completed
<ipython-input-45-04e17f2d375a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_l['growth_'+ str(i)+'d'] = df_l['Adj Close'] / df_l['Adj Close'].shift(i)
[*********************100%%**********************]  1 of 1 completed
<ipython-input-45-04e17f2d375a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_l['growth_'+ str(i)+'d'] = df_l['Adj Close'] / df_l['Adj Close'].shift(i)
[*********************100%%**********************]  1 of

,Open,High,Low,Close,Adj Close,Volume,Symbol,growth_7d
Date,,,,,,,,
2019-09-11,34.868782,35.511101,33.207928,34.038357,33.763878,14787563,PRX.AS,NaN
2019-09-12,34.501743,34.524685,32.978527,33.148285,32.880985,5842906,PRX.AS,NaN
2019-09-13,33.171223,34.042942,32.804184,33.905304,33.631897,5068221,PRX.AS,NaN
2019-09-16,33.515324,33.721783,32.340797,32.340797,32.080009,49689045,PRX.AS,NaN
2019-09-17,32.758305,33.033585,31.574600,31.794825,31.538439,10523710,PRX.AS,NaN
...,...,...,...,...,...,...,...,...
2024-04-24,3642.000000,3644.850098,3580.050049,3609.750000,3609.750000,652811,TITAN.NS,0.997361
2024-04-25,3595.000000,3602.500000,3541.000000,3571.100098,3571.100098,1889163,TITAN.NS,0.991807
2024-04-26,3588.000000,3593.250000,3556.300049,3584.800049,3584.800049,798783,TITAN.NS,0.983241


In [46]:
df_large_average = df_large.loc[df_large.index == '2014-01-01']
df_large_average.describe()

,Open,High,Low,Close,Adj Close,Volume,growth_7d
count,11.000000,11.000000,11.000000,11.000000,11.000000,1.100000e+01,11.000000
mean,379.070724,381.399366,376.303513,378.242693,343.090847,1.977886e+06,1.011684
std,487.881823,489.538106,484.275710,486.224155,453.133003,2.793953e+06,0.023147
min,40.433960,41.368938,40.204048,41.192673,37.967484,1.199500e+05,0.974774
25%,135.291668,138.520832,134.916668,136.098331,97.567871,4.822010e+05,0.996015
50%,230.000000,232.500000,229.350006,230.800003,220.363159,7.855360e+05,1.011587
75%,369.771683,370.402557,365.227753,366.122986,363.634323,1.410310e+06,1.026225
max,1770.000000,1776.900024,1756.550049,1764.000000,1626.533569,7.564701e+06,1.050544


In [47]:
df_largest_average = df_largest.loc[df_largest.index == '2014-01-01']
df_largest_average.describe()

,Open,High,Low,Close,Adj Close,Volume,growth_7d
count,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000e+00,7.000000
mean,496.876214,499.407041,492.764437,494.790582,423.456785,2.006836e+06,1.011797
std,325.140013,325.413385,320.594398,320.316271,258.021322,1.960422e+06,0.012192
min,177.199997,177.490005,176.125000,176.505005,160.800308,2.512040e+05,0.992906
25%,256.591415,260.563110,255.830910,259.666481,225.283997,6.121350e+05,1.006720
50%,410.283936,410.649689,405.780853,406.329468,379.456055,1.859278e+06,1.007909
75%,643.708344,647.166687,640.116669,642.358307,550.223404,2.404161e+06,1.019143
max,1090.050049,1092.250000,1075.550049,1076.650024,872.926331,5.904780e+06,1.030040


In [48]:
df_large_pivot = pd.pivot_table(df_large, values='growth_7d', index='Date', aggfunc = np.mean)
df_large_pivot

,growth_7d
Date,
2014-01-01,1.011684
2014-01-02,0.997706
2014-01-03,0.999534
2014-01-06,0.995876
2014-01-07,0.991592
...,...
2024-04-24,1.006752
2024-04-25,1.013231
2024-04-26,1.012366


In [49]:
df_largest_pivot = pd.pivot_table(df_largest, values='growth_7d', index='Date', aggfunc = np.mean)
df_largest_pivot

,growth_7d
Date,
2014-01-01,1.011797
2014-01-02,1.002964
2014-01-03,0.997668
2014-01-06,0.993423
2014-01-07,0.992509
...,...
2023-12-22,1.013788
2023-12-26,1.014900
2023-12-27,1.014155


In [50]:
df_merge = pd.merge(df_large_pivot, df_largest_pivot, how='left', left_index=True, right_index=True)
df_merge

,growth_7d_x,growth_7d_y
Date,,
2014-01-01,1.011684,1.011797
2014-01-02,0.997706,1.002964
2014-01-03,0.999534,0.997668
2014-01-06,0.995876,0.993423
2014-01-07,0.991592,0.992509
...,...,...
2024-04-24,1.006752,NaN
2024-04-25,1.013231,NaN
2024-04-26,1.012366,NaN


In [51]:
df_new = df_merge.rename(columns = {"growth_7d_x":"growth_7d_large", "growth_7d_y":"growth_7d_largest"})
df_new



,growth_7d_large,growth_7d_largest
Date,,
2014-01-01,1.011684,1.011797
2014-01-02,0.997706,1.002964
2014-01-03,0.999534,0.997668
2014-01-06,0.995876,0.993423
2014-01-07,0.991592,0.992509
...,...,...
2024-04-24,1.006752,NaN
2024-04-25,1.013231,NaN
2024-04-26,1.012366,NaN


In [52]:
df_new['large_greater_than_largest'] = (df_new['growth_7d_large'] > df_new['growth_7d_largest']).astype(int)
df_new

,growth_7d_large,growth_7d_largest,large_greater_than_largest
Date,,,
2014-01-01,1.011684,1.011797,0
2014-01-02,0.997706,1.002964,0
2014-01-03,0.999534,0.997668,1
2014-01-06,0.995876,0.993423,1
2014-01-07,0.991592,0.992509,0
...,...,...,...
2024-04-24,1.006752,NaN,0
2024-04-25,1.013231,NaN,0
2024-04-26,1.012366,NaN,0


In [53]:
sum_values = df_new['large_greater_than_largest'].sum()
f"the percentage of days that large stocks exceed largest stocks is {round(sum_values/2595*100,0)}%"

'the percentage of days that large stocks exceed largest stocks is 47.0%'

### Question 4: Trying Another Technical Indicators strategy

**What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?**


First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31).
If you encounter any difficulties running the Colab - you can download it using this [link](https://drive.google.com/file/d/1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb/view?usp=sharing).

Let's assume you've learned about the awesome **CCI indicator** ([Commodity Channel Index](https://www.investopedia.com/terms/c/commoditychannelindex.asp)), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (`Date.dt.dayofweek()==4`).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years?
One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit.
You need to sum these results over all trades (460 times in 10 years).

Additional:
  * Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
  * are you still profitable on those trades?

In [54]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz

!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=$HOME
!make
!make install

!TA_LIBRARY_PATH=~/lib TA_INCLUDE_PATH=~/include pip install ta-lib

--2024-05-09 18:38:52--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2024-05-09 18:38:52--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://deac-fra.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1 [following]
--2024-05-09 18:38:52--  http://deac-fra.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1
Resolvi

In [55]:
import talib

### [EXPLORATORY] Question 5: Finding Your Strategy for IPOs

You've seen in the first questions that the median and average investments are negative in IPOs, and you can't blindly invest in all deals.

How would you correct/refine the approach? Briefly describe the steps and the data you'll try to get (it should be generally feasible to do it from public sources - no access to internal data of companies)?

E.g. (some ideas) Do you want to focus on the specific vertical? Do you want to build a smart comparison vs. existing stocks on the market? Or you just will want to get some features (which features?) like total number of people in a company to find a segment of "successful" IPOs?

---

In [56]:
!pip3 install bs4

In [57]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url_sp400= "https://en.wikipedia.org/wiki/List_of_S%26P_400_companies"
response = requests.get(url_sp400, headers=headers)

df_sp400 = pd.read_html(response.text)
df_sp400[0]

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,SEC filings
0,AA,Alcoa,Materials,Aluminum,"Pittsburgh, Pennsylvania",reports
1,ACHC,Acadia Healthcare,Health Care,Health Care Facilities,"Franklin, Tennessee",reports
2,ACM,AECOM,Industrials,Construction & Engineering,"Dallas, Texas",reports
3,ADC,Agree Realty,Real Estate,Retail REITs,"Bloomfield Hills, Michigan",reports
4,ADNT,Adient,Consumer Discretionary,Automotive Parts & Equipment,"Plymouth, Michigan",reports
...,...,...,...,...,...,...
396,XRAY,Dentsply Sirona,Health Care,Health Care Supplies,"Charlotte, North Carolina",reports
397,YETI,Yeti,Consumer Discretionary,Leisure Products,"Austin, Texas",reports
398,ZD,Ziff Davis,Communication Services,Advertising,"New York City, New York",reports
399,ZI,ZoomInfo,Communication Services,Interactive Media & Services,"Vancouver, Washington",reports


In [58]:
from bs4 import BeautifulSoup

url_sp500= "https://r2.datahub.io/clv15437h0003mj091e4korcs/main/raw/data/constituents.csv"
data = requests.get(url_sp500).text
data

'Symbol,Name,Sector\nMMM,3M Company,Industrials\nAOS,A.O. Smith Corp,Industrials\nABT,Abbott Laboratories,Health Care\nABBV,AbbVie Inc.,Health Care\nACN,Accenture plc,Information Technology\nATVI,Activision Blizzard,Information Technology\nAYI,Acuity Brands Inc,Industrials\nADBE,Adobe Systems Inc,Information Technology\nAAP,Advance Auto Parts,Consumer Discretionary\nAMD,Advanced Micro Devices Inc,Information Technology\nAES,AES Corp,Utilities\nAET,Aetna Inc,Health Care\nAMG,Affiliated Managers Group Inc,Financials\nAFL,AFLAC Inc,Financials\nA,Agilent Technologies Inc,Health Care\nAPD,Air Products & Chemicals Inc,Materials\nAKAM,Akamai Technologies Inc,Information Technology\nALK,Alaska Air Group Inc,Industrials\nALB,Albemarle Corp,Materials\nARE,Alexandria Real Estate Equities Inc,Real Estate\nALXN,Alexion Pharmaceuticals,Health Care\nALGN,Align Technology,Health Care\nALLE,Allegion,Industrials\nAGN,"Allergan, Plc",Health Care\nADS,Alliance Data Systems,Information Technology\nLNT,Alli

In [59]:
!curl -o sp500.csv https://r2.datahub.io/clv15437h0003mj091e4korcs/main/raw/data/constituents.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18676  100 18676    0     0   118k      0 --:--:-- --:--:-- --:--:--  118k


In [60]:
import yfinance as yf

msft = yf.Ticker("MSFT")
msft.income_stmt

,2023-06-30,2022-06-30,2021-06-30,2020-06-30
Tax Effect Of Unusual Items,-2850000.0,43420000.0,180160797.164637,4760000.0
Tax Rate For Calcs,0.19,0.13,0.138266,0.17
Normalized EBITDA,105155000000.0,99905000000.0,83831000000.0,68395000000.0
Total Unusual Items,-15000000.0,334000000.0,1303000000.0,28000000.0
Total Unusual Items Excluding Goodwill,-15000000.0,334000000.0,1303000000.0,28000000.0
Net Income From Continuing Operation Net Minority Interest,72361000000.0,72738000000.0,61271000000.0,44281000000.0
Reconciled Depreciation,13861000000.0,14460000000.0,11686000000.0,12796000000.0
Reconciled Cost Of Revenue,65863000000.0,62650000000.0,52232000000.0,46078000000.0
EBITDA,105140000000.0,100239000000.0,85134000000.0,68423000000.0
EBIT,91279000000.0,85779000000.0,73448000000.0,55627000000.0


In [61]:
ticker = 'MSFT'
df_ticker = yf.download(tickers = ticker,
                        period = "max",
                        interval = "1d")
df_ticker['Symbol'] = ticker

[*********************100%%**********************]  1 of 1 completed


In [62]:
ms = msft.income_stmt
table = pd.pivot_table(data = ms, index=['2020-06-30'])
table

,2021-06-30,2022-06-30,2023-06-30
2020-06-30 00:00:00,,,
-1.330000e+08,-13000000.0,-101000000.0,-30000000.0
-4.000000e+07,98000000.0,-32000000.0,-223000000.0
-1.200000e+07,1401000000.0,302000000.0,-238000000.0
1.700000e-01,0.138266,0.13,0.19
5.760000e+00,8.05,9.65,9.68
5.820000e+00,8.12,9.7,9.72
4.760000e+06,180160797.164637,43420000.0,-2850000.0
2.800000e+07,1303000000.0,334000000.0,-15000000.0
8.900000e+07,-215000000.0,31000000.0,1026000000.0


In [63]:
msft.income_stmt

,2023-06-30,2022-06-30,2021-06-30,2020-06-30
Tax Effect Of Unusual Items,-2850000.0,43420000.0,180160797.164637,4760000.0
Tax Rate For Calcs,0.19,0.13,0.138266,0.17
Normalized EBITDA,105155000000.0,99905000000.0,83831000000.0,68395000000.0
Total Unusual Items,-15000000.0,334000000.0,1303000000.0,28000000.0
Total Unusual Items Excluding Goodwill,-15000000.0,334000000.0,1303000000.0,28000000.0
Net Income From Continuing Operation Net Minority Interest,72361000000.0,72738000000.0,61271000000.0,44281000000.0
Reconciled Depreciation,13861000000.0,14460000000.0,11686000000.0,12796000000.0
Reconciled Cost Of Revenue,65863000000.0,62650000000.0,52232000000.0,46078000000.0
EBITDA,105140000000.0,100239000000.0,85134000000.0,68423000000.0
EBIT,91279000000.0,85779000000.0,73448000000.0,55627000000.0


In [64]:

list = ['SPPL', 'MSS', 'HLP', 'VSME', 'PMEC', 'MRM']


In [65]:
ticker = "^GSPC"
df_i = yf.Ticker(ticker)
df_i


yfinance.Ticker object <^GSPC>

In [66]:
#list = ['MRM', 'VHAQ', 'IKT']

In [67]:
date_list, revenue, ticker_name, gp, eps = [], [], [], [], []
for stock in list:
  df = yf.Ticker(stock)
  for i in df.income_stmt.columns:
    date_list.append(i)
    revenue.append(df.income_stmt[i]['Total Revenue']/1000000)
    ticker_name.append(stock)
    try:
      gp.append(df.income_stmt[i]['Gross Profit']/1000000)
    except ValueError as e:
      f"stock: {stock} has no gross profit value: {e}"
    eps.append(df.income_stmt[i]['Basic EPS'])
s = pd.DataFrame(columns = ['Date', 'Ticker', 'Revenue', 'Gross Profit', 'Basic EPS'], data = zip(date_list, ticker_name, revenue, gp, eps))
s['Gross Profit Margin'] = s['Gross Profit']/s['Revenue']
s


,Date,Ticker,Revenue,Gross Profit,Basic EPS,Gross Profit Margin
0,2022-12-31,SPPL,6.510169,3.599296,NaN,0.552873
1,2021-12-31,SPPL,4.179300,2.326549,0.004097,0.556684
2,2020-12-31,SPPL,3.544396,1.936243,-0.025731,0.546283
3,2023-04-30,MSS,55.399112,12.451160,0.067665,0.224754
4,2022-04-30,MSS,41.984221,8.286624,-0.030419,0.197375
5,2021-04-30,MSS,41.195276,8.310502,0.048869,0.201734
6,2023-12-31,HLP,15.997954,5.245840,NaN,0.327907
7,2022-12-31,HLP,20.283245,7.008493,0.243097,0.345531
8,2021-12-31,HLP,21.713138,7.654308,0.265468,0.352520
9,2020-12-31,HLP,11.158820,4.452517,0.200948,0.399013


In [68]:
subset = []
for i in list:
  for ticker in s['Ticker'].unique():
    if i == ticker:
       ss = s.loc[s.Ticker == ticker]
       ss['Revenue_minus_1'] = ss['Revenue'].shift(-1)
       ss["Revenue_growth_1y"] = ss['Revenue']/ss['Revenue_minus_1'] - 1
       ss['GPM_minus_1'] = ss['Gross Profit Margin'].shift(-1)
       ss["GPM_growth_1y"] = ss['Gross Profit Margin']/ss['GPM_minus_1'] - 1
       ss['Basic_EPS_minus_1'] = ss['Basic EPS'].shift(-1)
       ss["Basic_EPS_growth_1y"] = ss['Basic EPS']/ss['Basic_EPS_minus_1'] - 1
       subset.append(ss)
  subset

<ipython-input-68-400b44b767fb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ss['Revenue_minus_1'] = ss['Revenue'].shift(-1)
<ipython-input-68-400b44b767fb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ss["Revenue_growth_1y"] = ss['Revenue']/ss['Revenue_minus_1'] - 1
<ipython-input-68-400b44b767fb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [69]:
df_subset = pd.concat(subset, axis=0, ignore_index=False)
df_subset.drop(columns = ['Revenue_minus_1', 'GPM_minus_1', 'Basic_EPS_minus_1'], inplace=True)
display(df_subset)


,Date,Ticker,Revenue,Gross Profit,Basic EPS,Gross Profit Margin,Revenue_growth_1y,GPM_growth_1y,Basic_EPS_growth_1y
0,2022-12-31,SPPL,6.510169,3.599296,NaN,0.552873,0.557718,-0.006846,NaN
1,2021-12-31,SPPL,4.179300,2.326549,0.004097,0.556684,0.179129,0.019040,-1.159224
2,2020-12-31,SPPL,3.544396,1.936243,-0.025731,0.546283,NaN,NaN,NaN
3,2023-04-30,MSS,55.399112,12.451160,0.067665,0.224754,0.319522,0.138716,-3.224432
4,2022-04-30,MSS,41.984221,8.286624,-0.030419,0.197375,0.019151,-0.021611,-1.622460
5,2021-04-30,MSS,41.195276,8.310502,0.048869,0.201734,NaN,NaN,NaN
6,2023-12-31,HLP,15.997954,5.245840,NaN,0.327907,-0.211272,-0.051006,NaN
7,2022-12-31,HLP,20.283245,7.008493,0.243097,0.345531,-0.065854,-0.019824,-0.084270
8,2021-12-31,HLP,21.713138,7.654308,0.265468,0.352520,0.945827,-0.116521,0.321078
9,2020-12-31,HLP,11.158820,4.452517,0.200948,0.399013,NaN,NaN,NaN
